<a href="https://colab.research.google.com/github/zflink/Nitrospira_enrichments/blob/main/Plotting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nitrospira enrichments growth curve plotting

In [50]:
#import python packages
import pandas as pd
import numpy as np
import altair as alt
from altair import datum

In [18]:
#'https://docs.google.com/spreadsheets/d/1USvlnH7I8knQDQKhLYuWhanpWq0NiRlE5-xYncjNrDQ/edit#gid=0'

sheet_ID = "1USvlnH7I8knQDQKhLYuWhanpWq0NiRlE5-xYncjNrDQ"
xls = pd.ExcelFile(f"https://docs.google.com/spreadsheets/d/{sheet_ID}/export?format=xlsx")
df = pd.read_excel(xls,"Sheet1",header = 0)
df.head()

,Date,Organism,Treatment,Start_date,NH4_ug-N-L,Nitrite_ug-N-L,TON_ug-N-L,Cell_count_dilution,Cell_count_volume,Time_elapsed_hr,Cell_count,NH4_uM,Nitrite_uM,TON_uM,Nitrate_uM
0,2024-04-04 15:00:00,R1w,Vitamins,2024-04-04 15:00:00,0.86,13580.72,15251,NaN,NaN,0.0,NaN,0.061398,969.566645,1088.812736,119.246091
1,2024-04-04 15:00:00,R1w2,Vitamins,2024-04-04 15:00:00,0.53,13194.85,13633,NaN,NaN,0.0,NaN,0.037838,942.018277,973.299065,31.280788
2,2024-04-08 12:00:00,R1w,Vitamins,2024-04-04 15:00:00,72.66,13155.69,15458,2.0,6.37,93.0,1569858.713,5.187406,939.222532,1103.591062,164.368530
3,2024-04-08 12:00:00,R1w2,Vitamins,2024-04-04 15:00:00,68.47,12711.90,14062,2.0,5.35,93.0,1869158.879,4.888270,907.539088,1003.926608,96.387521
4,2024-04-08 15:00:00,R1w,No_Vitamins,2024-04-08 15:00:00,0.00,13599.42,15566,NaN,NaN,0.0,NaN,0.000000,970.901692,1111.301492,140.399800


In [101]:
#Define base timeseries chart format
base = alt.Chart(data=df).mark_line(point=True, opacity=0.3).encode(
            alt.X('Time_elapsed_hr:Q'),
            alt.Color('Treatment:N'),
            alt.Shape('Organism:N')).properties(height=200, width=300)

In [ ]:
#Define function to filter dataset and plot specified parameter on y-axis
def filter_plot(base, treatment, organism, date, y):
  return base.transform_filter(
          (datum.Treatment==treatment) & (datum.Organism==organism) & (datum.Start_date==date)
          ).encode(alt.Y(y))

In [105]:
charts_cell_count = []
charts_nitrite = []
charts_nitrate = []
charts_ammonia = []

for treatment in set(df.Treatment):
  for organism in set(df.Organism):
    subset = df[(df.Treatment == treatment) & (df.Organism == organism)]
    for start_date in set(subset.Start_date):
      formatted_date = str(start_date).split()[0]+'T'+str(start_date).split()[1]
      charts_cell_count.append(filter_plot(base, treatment, organism, formatted_date, 'Cell_count:Q'))
      charts_nitrite.append(filter_plot(base, treatment, organism, formatted_date, 'Nitrite_uM:Q'))
      charts_nitrate.append(filter_plot(base, treatment, organism, formatted_date, 'Nitrate_uM:Q'))
      charts_ammonia.append(filter_plot(base, treatment, organism, formatted_date, 'NH4_uM:Q'))

alt.layer(*charts_cell_count) & alt.layer(*charts_nitrite) | alt.layer(*charts_nitrate) & alt.layer(*charts_ammonia)

alt.HConcatChart(...)